In [1]:
import pytorch_lightning as pl
import torch.nn as nn

from torch.utils.data import DataLoader
from CFS_SG import CFS_SG

import torch
from torch.utils.data import TensorDataset

In [2]:
import matplotlib
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import svd
from scipy.linalg import pinv
from itertools import product
import numpy as np, h5py, os
import matplotlib.pyplot as plt
from operator import itemgetter 
from scipy.sparse import vstack, coo_matrix, csc_matrix, isspmatrix_csc
%matplotlib inline
import scanpy as sc
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [3]:
target = pd.read_csv("small_molecule_fore.csv", index_col = 0)
background = pd.read_csv("small_molecule_back.csv", index_col = 0)
gene_names = target.columns.to_numpy()
cell_names = target.index.to_numpy()

In [4]:
labels_train = np.concatenate([np.zeros(background.shape[0]), np.ones(target.shape[0])])
data_train = np.concatenate([background, target])

data_train_tensor = torch.tensor(data_train, dtype=torch.float32)  
labels_train_tensor = torch.tensor(labels_train, dtype=torch.long) 

dataset = TensorDataset(data_train_tensor, labels_train_tensor)


In [5]:
input_size = target.shape[1]
output_size = background.shape[1]
batch_size = 128

In [6]:
model = CFS_SG(
    input_size=input_size,
    output_size=output_size,
    hidden=[512, 512], # Number of units in each hidden layer
    k_prime=20, # Background dimension size
    lam=0.175, # Tuned to select about 10 features
    lr=1e-3,
    loss_fn=nn.MSELoss()
)


In [7]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

trainer = pl.Trainer(max_epochs=10, accelerator='gpu', devices=1)
trainer.fit(model, loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/nas/longleaf/home/eyzhang/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 2g.10gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.htm

Training: |          | 0/? [00:00<?, ?it/s]

/nas/longleaf/home/eyzhang/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: You called `self.log('num_selected_gates', ...)` in your `training_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'num_selected_gates': ...})` instead.
`Trainer.fit` stopped: `max_epochs=10` reached.


In [8]:
trainer = pl.Trainer(max_epochs=10, accelerator='gpu', devices=1)
trainer.fit(model, loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-f431a4cc-e3b9-502a-a0e9-8bb3d21b4074]

  | Name                   | Type       | Params | Mode 
--------------------------------------------------------------
0 | background_input_layer | Sequential | 130 K  | train
1 | fc                     | Sequential | 1.3 M  | train
2 | loss_fn                | MSELoss    | 0      | train
  | other params           | n/a        | 1.0 K  | n/a  
--------------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.720     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [9]:
indices = model.get_inds(10) 

In [10]:
gene_names[indices]

array(['ENSG00000169715', 'ENSG00000198959', 'ENSG00000245532',
       'ENSG00000196754', 'ENSG00000168036', 'ENSG00000123689',
       'ENSG00000197061', 'ENSG00000135679', 'ENSG00000169429',
       'ENSG00000205426'], dtype=object)